In [1]:
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
import os
import cv2
import wandb
from datetime import datetime
from tqdm import tqdm

from dataset.aptos import APTOS

import torch
from torch.utils.data import Dataset, DataLoader

# Which GPU to use
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [2]:
aptos_train = APTOS('/ssd_Samsung870_2T/hc701-fed/hc701-project')
aptos_train_loader = DataLoader(aptos_train, batch_size=8, shuffle=True, num_workers=0)

### DensnNet121

In [3]:
# DenseNet121 baseline
from model.baseline import Baseline
DesenNet121 = Baseline('densenet121', 5, pretrained=False)
Loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(DesenNet121.parameters(), lr=0.0001)
run = wandb.init(project="hc701-fed", entity="arcticfox", name=f"baseline-densenet121-{str(datetime.now())}",job_type='train',reinit=True,)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arctic (arcticfox). Use `wandb login --relogin` to force relogin


In [4]:
# Train
DesenNet121.to(device)
DesenNet121.train()
for epoch in tqdm(range(10)):
    for i, (image, label) in enumerate(aptos_train_loader):
        image = image.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = DesenNet121(image)
        loss = Loss(output, label)
        run.log({"loss": loss.cpu().detach().numpy()})
        loss.backward()
        optimizer.step()
run.finish()

 20%|██        | 2/10 [32:12<2:08:52, 966.51s/it]

In [ ]:
# Test in the training set
# Out put the accuracy and f1 score
from sklearn.metrics import f1_score
DesenNet121.eval()
correct = 0
total = 0
predict_result = []
true_result = []
with torch.no_grad():
    for i, (image, label) in enumerate(aptos_train_loader):
        image = image.to(device)
        label = label.to(device)
        output = DesenNet121(image)
        _, predicted = torch.max(output.data, 1)
        predict_result.append(predicted)
        true_result.append(label.cpu().numpy().tolist())
        total += label.size(0)
        correct += (predicted == label).sum().item()
print('Accuracy of the network on the train images: %d %%' % (100 * correct / total))

In [ ]:
y_pre = []
for i in predict_result:
    j  = i.cpu().numpy().tolist()
    for k in j:
        y_pre.append(k)

In [ ]:
y_true = []
for i in true_result:
    for j in i:
        y_true.append(j)

In [ ]:
f1_score(y_true, y_pre, average='macro')